In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import scanpy as sc
from muon import MuData
import muon as mu

/home/brent10070/miniconda3/envs/Transformer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read Data

In [46]:
DataPathway = ""

In [29]:
filePathway = "%sshare.h5mu" %DataPathway
mdata = mu.read(filePathway)

In [3]:
mdata

MuData object with n_obs × n_vars = 34500 × 356877
  obs:	'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'celltype'
  2 modalities
    RNA:	34500 x 22537
      layers:	'counts', 'data'
    ATAC:	34500 x 334340
      var:	'count', 'percentile'
      layers:	'counts'

### Split Datasets

##### Train : Validation : Test = 70% : 10% : 20%

In [30]:
Train_Validation_barcode, Test_barcode =  train_test_split(mdata.obs_names, test_size =.20, shuffle  = True) 
Train_barcode, Validation_barcode =  train_test_split(Train_Validation_barcode, test_size =.125, shuffle  = True) 
len(Train_barcode), len(Validation_barcode), len(Test_barcode)

(24150, 3450, 6900)

In [31]:
Train = mdata[Train_barcode]
Validation = mdata[Validation_barcode]
Test = mdata[Test_barcode]

## Train

### PCA for scRNA-seq and Get the 30 Neighbors

In [32]:
sc.tl.pca(Train.mod['RNA'])
sc.pp.neighbors(Train.mod['RNA'], n_neighbors=31, n_pcs=30)
Train.mod['RNA']

/home/brent10070/miniconda3/envs/Transformer/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:229: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata.obsm['X_pca'] = X_pca


AnnData object with n_obs × n_vars = 24150 × 22537
    uns: 'pca', 'neighbors'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'data'
    obsp: 'distances', 'connectivities'

In [33]:
Distance_index = Train.mod['RNA'].obsp['distances'].toarray().argsort(axis=1)[:, -30:]
col_names = ["Neigh_{}".format(i) for i in range(1, 31)]
Train_knn_neighbor = pd.DataFrame(Distance_index, columns=col_names)
Train_knn_neighbor.index = [Train.mod['RNA'].obs_names]
Train_knn_neighbor

,Neigh_1,Neigh_2,Neigh_3,Neigh_4,Neigh_5,Neigh_6,Neigh_7,Neigh_8,Neigh_9,Neigh_10,...,Neigh_21,Neigh_22,Neigh_23,Neigh_24,Neigh_25,Neigh_26,Neigh_27,Neigh_28,Neigh_29,Neigh_30
R1.14.R2.53.R3.32.P1.07,3167,10426,6684,5828,9834,20794,2544,8466,21818,4280,...,5774,1945,187,6611,15367,7409,20876,19207,1629,16177
R1.73.R2.83.R3.67.P1.08,23946,11768,6476,8210,1258,5449,6957,22280,22080,2246,...,7605,16274,2406,4285,21395,8300,21942,21947,2305,678
R1.51.R2.10.R3.26.P1.06,9352,15144,11976,16963,20257,19459,7130,21393,16295,13782,...,6492,6755,14137,12521,14779,4051,19671,6837,8406,13784
R1.75.R2.73.R3.35.P1.06,13352,23825,15681,4022,9519,18109,9649,10418,15097,17335,...,15874,14459,20958,8914,4520,3230,20531,12133,11756,6473
R1.69.R2.42.R3.36.P1.05,1266,22446,12687,16940,12943,14586,8175,13505,8409,7188,...,8297,7002,13386,17932,16034,14405,6902,19901,18188,7048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R1.39.R2.68.R3.44.P1.05,17912,2873,5517,15476,4274,14466,7614,14954,21742,23874,...,1068,20980,13931,16916,12089,5790,5347,12983,6863,7201
R1.70.R2.18.R3.02.P1.08,5349,20331,1511,6189,10287,8550,5400,8675,14451,17448,...,14795,4992,15486,22043,11433,19557,19080,16953,3717,11574
R1.95.R2.02.R3.74.P1.08,437,14701,3700,18418,9182,11825,11725,11598,18677,11256,...,20900,21361,5776,15295,2942,113,10770,11730,6671,1878
R1.47.R2.54.R3.35.P1.06,19292,20677,6771,3207,14042,5088,7926,1466,20022,442,...,3785,9389,5100,10107,14456,4206,18987,7815,337,1183


### Get 30 neighbors expression values of the cell

In [14]:
sc.get.var_df(Train.mod['RNA'], keys=Train.mod['RNA'].obs_names[Train_knn_neighbor.iloc[6]].tolist())

,R1.02.R2.32.R3.15.P1.05,R1.48.R2.47.R3.21.P1.05,R1.77.R2.84.R3.03.P1.05,R1.45.R2.91.R3.72.P1.05,R1.08.R2.76.R3.89.P1.05,R1.19.R2.59.R3.28.P1.05,R1.16.R2.70.R3.68.P1.05,R1.64.R2.20.R3.33.P1.05,R1.15.R2.11.R3.41.P1.05,R1.96.R2.44.R3.40.P1.05,...,R1.07.R2.07.R3.43.P1.05,R1.14.R2.76.R3.60.P1.05,R1.96.R2.69.R3.71.P1.05,R1.03.R2.39.R3.74.P1.05,R1.18.R2.35.R3.74.P1.05,R1.22.R2.08.R3.39.P1.05,R1.65.R2.45.R3.68.P1.05,R1.52.R2.90.R3.77.P1.05,R1.44.R2.02.R3.01.P1.05,R1.24.R2.68.R3.69.P1.05
0610007P14Rik,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,...,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023,-0.093023
0610009B22Rik,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,...,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560,-0.049560
0610009L18Rik,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,...,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161,-0.035161
0610009O20Rik,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,...,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182,-0.119182
0610010F05Rik,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,...,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639,-0.167639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,...,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384,-0.005384
mt-Nd4,-0.293290,-0.293290,-0.293290,-0.293290,-0.293290,-0.293290,1.582112,-0.293290,-0.293290,-0.293290,...,0.328725,-0.293290,-0.293290,0.478744,-0.293290,-0.293290,-0.293290,-0.293290,-0.293290,-0.293290
mt-Nd4l,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,...,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240,-0.007240
mt-Nd5,-0.475177,0.096704,-0.475177,-0.475177,-0.475177,0.284277,-0.475177,-0.475177,0.151333,-0.021418,...,-0.100661,0.434327,0.751395,-0.475177,-0.475177,-0.475177,0.203070,0.421054,0.002485,-0.475177


## Test

In [35]:
sc.tl.pca(Test.mod['RNA'])
sc.pp.neighbors(Test.mod['RNA'], n_neighbors=31, n_pcs=30)
Test.mod['RNA']

/home/brent10070/miniconda3/envs/Transformer/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:229: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata.obsm['X_pca'] = X_pca


AnnData object with n_obs × n_vars = 6900 × 22537
    uns: 'pca', 'neighbors'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'data'
    obsp: 'distances', 'connectivities'

In [36]:
Distance_index = Test.mod['RNA'].obsp['distances'].toarray().argsort(axis=1)[:, -30:]
col_names = ["Neigh_{}".format(i) for i in range(1, 31)]
Test_knn_neighbor = pd.DataFrame(Distance_index, columns=col_names)
Test_knn_neighbor.index = [Test.mod['RNA'].obs_names]

## Validation

In [38]:
sc.tl.pca(Validation.mod['RNA'])
sc.pp.neighbors(Validation.mod['RNA'], n_neighbors=31, n_pcs=30)
Validation.mod['RNA']

/home/brent10070/miniconda3/envs/Transformer/lib/python3.9/site-packages/scanpy/preprocessing/_pca.py:229: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata.obsm['X_pca'] = X_pca


AnnData object with n_obs × n_vars = 3450 × 22537
    uns: 'pca', 'neighbors'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts', 'data'
    obsp: 'distances', 'connectivities'

In [39]:
Distance_index = Validation.mod['RNA'].obsp['distances'].toarray().argsort(axis=1)[:, -30:]
col_names = ["Neigh_{}".format(i) for i in range(1, 31)]
Validation_knn_neighbor = pd.DataFrame(Distance_index, columns=col_names)
Validation_knn_neighbor.index = [Validation.mod['RNA'].obs_names]

### Save object

In [42]:
Train.write("%sdatasets/Train.h5mu"%DataPathway)
Test.write("%sdatasets/Test.h5mu"%DataPathway)
Validation.write("%sdatasets/Validation.h5mu"%DataPathway)

### Save Neighbor Table

In [61]:
Train_knn_neighbor.to_csv("%sdatasets/Train_neighbor.csv"%DataPathway, index=True)
Test_knn_neighbor.to_csv("%sdatasets/Test_neighbor.csv"%DataPathway, index=True)
Validation_knn_neighbor.to_csv("%sdatasets/Validation_neighbor.csv"%DataPathway, index=True)